# SNN

--------------------------
## Personal
#### Things to Do:
- (DONE) L2 normalisation gives slower training so that line has been commented out.Need to figure out why that's happening and if it is normal
- (DONE) Implement modified softmax
- Figure out class weights and incorporate that
- check if alpha needs a relu
- Batching the training
- Logging information nicely
--------------------------

## Resources and Information

Additive margin loss: https://arxiv.org/pdf/1801.05599.pdf
<br>Large margin softmax: https://arxiv.org/pdf/1612.02295.pdf https://github.com/auroua/L_Softmax_TensorFlow/blob/master/nets/l_softmax.py

--------------------------

## Code
--------------------------

In [156]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.datasets import mnist
from sklearn.preprocessing import normalize

### MNIST Data

In [6]:
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()
x_train_mnist = x_train_mnist.reshape(x_train_mnist.shape[0], 784)
x_test_mnist = x_test_mnist.reshape(x_test_mnist.shape[0], 784)
x_train_mnist = x_train_mnist.astype('float32')
x_test_mnist = x_test_mnist.astype('float32')
x_train_mnist /= 255
x_test_mnist /= 255

data_size = 5000

X_mnist = x_train_mnist[:data_size]
y_mnist = y_train_mnist[:data_size]

y_onehot_mnist = np.zeros([data_size, 10])
y_onehot_test_mnist = np.zeros([len(y_test_mnist), 10])

for a in range(data_size):
    y_onehot_mnist[a][y_mnist[a]] = 1
for a in range(len(y_test_mnist)):
    y_onehot_test_mnist[a][y_test_mnist[a]] = 1

### Network Parameters

In [7]:
input_size = 28*28
n_layers = 10
n_classes = 10
n_units = 30
batch_size = 50
epochs = 50
learning_rate = 0.05

### Accuracy Function

In [12]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

### Creating the Network

In [32]:
tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, input_size])
original_input = inputs
labels = tf.placeholder(tf.float32, [None, n_classes])

for a in range(n_layers):
    layer = tf.layers.dense(inputs, n_units, 'selu', name='layer'+str(a))
    inputs = tf.concat([inputs, layer], 1, name='concatenation'+str(a))

logits = tf.layers.dense(inputs, n_classes, None, name='output')
logits = tf.math.l2_normalize(logits, axis=1)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

predictions = tf.nn.softmax(logits)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

### Running the Network

In [33]:
with tf.Session() as session:
    tf.initialize_all_variables().run()
    print("Initialized")

    for a in range(epochs):
        _, l, p = session.run([optimizer, loss, predictions], 
                              feed_dict={original_input:X_mnist, labels:y_onehot_mnist})

        if a%50==0 or 1:
            print("Epoch:", a+1, "\tLoss:", l, "\tAccuracy:", accuracy(p, y_onehot_mnist))

    test_p = session.run(predictions, feed_dict={original_input:x_test_mnist, labels:y_onehot_test_mnist})

    print("Test Accuracy: ", accuracy(test_p, y_onehot_test_mnist))

    
    tf.summary.FileWriter('./logs', session.graph)

Initialized
Epoch: 1 	Loss: 2.3350246 	Accuracy: 15.48
Epoch: 2 	Loss: 2.2935538 	Accuracy: 19.54
Epoch: 3 	Loss: 2.2508812 	Accuracy: 23.78
Epoch: 4 	Loss: 2.2049522 	Accuracy: 27.44
Epoch: 5 	Loss: 2.1543813 	Accuracy: 31.68
Epoch: 6 	Loss: 2.0997586 	Accuracy: 35.34
Epoch: 7 	Loss: 2.04551 	Accuracy: 40.8
Epoch: 8 	Loss: 1.9983243 	Accuracy: 46.66
Epoch: 9 	Loss: 1.9609023 	Accuracy: 51.52
Epoch: 10 	Loss: 1.9313827 	Accuracy: 55.56
Epoch: 11 	Loss: 1.9074184 	Accuracy: 59.22
Epoch: 12 	Loss: 1.8875211 	Accuracy: 62.32
Epoch: 13 	Loss: 1.8707762 	Accuracy: 64.78
Epoch: 14 	Loss: 1.8565377 	Accuracy: 66.74
Epoch: 15 	Loss: 1.8443071 	Accuracy: 68.5
Epoch: 16 	Loss: 1.8336918 	Accuracy: 69.62
Epoch: 17 	Loss: 1.8243876 	Accuracy: 70.74
Epoch: 18 	Loss: 1.8161559 	Accuracy: 71.6
Epoch: 19 	Loss: 1.8088094 	Accuracy: 72.58
Epoch: 20 	Loss: 1.802201 	Accuracy: 73.5
Epoch: 21 	Loss: 1.7962136 	Accuracy: 74.46
Epoch: 22 	Loss: 1.7907518 	Accuracy: 75.26
Epoch: 23 	Loss: 1.785741 	Accuracy:

### Trial for Loss

In [229]:
embedding_length = 12
number_of_samples = 22
number_of_classes = 27

tf.reset_default_graph()

labels = tf.placeholder(tf.int32, [None], name='labels')

embeddings = tf.placeholder(tf.float32, [None, embedding_length])
norm_embeddings = tf.nn.l2_normalize(embeddings, axis=1, name='embeddings')

class_weights = tf.constant(normalize(np.random.rand(embedding_length, number_of_classes)), dtype=tf.float32,
                           name='class_weights')

margin = tf.placeholder(tf.float32, name='margin')

alpha_initial = tf.get_variable(dtype=tf.float32, initializer=tf.constant(np.random.rand(1), dtype='float32'), name='alpha')
alpha = tf.nn.relu(alpha_initial)

cosines = tf.matmul(norm_embeddings, class_weights, name='cosines')
onehot_label = tf.one_hot(labels, number_of_classes, name='labels_onehot')
m_onehot = tf.math.multiply(margin, onehot_label, name='mxonehot')
margin_cosine = tf.subtract(cosines, m_onehot, name='cosine-m')
alpha_margin_cosine = tf.math.exp(tf.math.multiply(alpha, margin_cosine), name='alphaxc-m')
amc_numerator = tf.reduce_sum(tf.multiply(alpha_margin_cosine, onehot_label), axis=1, name='amc_positive')
amc_denominator = tf.reduce_sum(alpha_margin_cosine, axis=1, name='amc_total')
amc_fraction = tf.divide(amc_numerator, amc_denominator, name='amc_fraction')
log_amcf = -tf.log(amc_fraction, name='log_amc')
loss = tf.reduce_sum(log_amcf, name='loss')

m = 0
m_change = 0.00002
m_max = 25

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as session:
    feed_dict={embeddings:np.random.rand(number_of_samples, embedding_length),
                                     labels: np.random.randint(0, number_of_classes, number_of_samples),
                                      margin: m}
    alpha_initial.initializer.run()
    
    for a in range(1000):
        l, o, a = session.run([loss, optimizer, alpha], feed_dict=feed_dict)
        print(l, o, a)
        feed_dict[margin] = min(feed_dict[margin]+m_change, m_max)
    tf.summary.FileWriter('./logs', session.graph)

72.586655 None [0.7523083]
72.58588 None [0.7448853]
72.5851 None [0.73748493]
72.58432 None [0.7301071]
72.58354 None [0.72275174]
72.58277 None [0.7154186]
72.58201 None [0.7081076]
72.581245 None [0.70081866]
72.58049 None [0.6935516]
72.579735 None [0.68630624]
72.57898 None [0.67908245]
72.57823 None [0.67188025]
72.577484 None [0.6646994]
72.576744 None [0.6575398]
72.576004 None [0.6504013]
72.57526 None [0.6432837]
72.574524 None [0.6361871]
72.5738 None [0.6291112]
72.57307 None [0.62205595]
72.572334 None [0.61502117]
72.57162 None [0.6080068]
72.5709 None [0.6010126]
72.570175 None [0.5940386]
72.569466 None [0.58708465]
72.56875 None [0.58015054]
72.56804 None [0.5732362]
72.56734 None [0.5663416]
72.56663 None [0.55946654]
72.56592 None [0.5526109]
72.56522 None [0.5457746]
72.56452 None [0.5389575]
72.56383 None [0.53215945]
72.56314 None [0.5253804]
72.56244 None [0.51862025]
72.56175 None [0.51187885]
72.561066 None [0.5051561]
72.56038 None [0.4984519]
72.55969 None [0

72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
72.508415 None [0.]
